In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data download
# kaggle: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

!wget -q --show-progress https://raw.githubusercontent.com/rambabar/NLP/main/data/spam.csv

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.03s   


In [ ]:
# # If Training time of model is more to prevent disconnect
# #Just open Chrome DevTools by pressing F12 or Ctrl+Shift+I on Linux & run following JavaScript code in your console:
# '''
# function KeepClicking(){
#    console.log("Clicking");
#    document.querySelector("colab-toolbar-button#connect").click()
# }setInterval(KeepClicking,60000)
# '''

In [ ]:
# #Spark written in Scala programming language & requires Java Virtual Machine (JVM) to run. Therefore, download Java.
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # install Apache Spark with Hadoop (change the version number if needed) # https://downloads.apache.org/spark/
# # !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# !wget -q http://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# !tar xf spark-3.3.1-bin-hadoop3.tgz

# # unzip the spark file to the current folder
# # !tar xf spark-3.0.0-bin-hadoop3.2.tgz
# #install findspark library using pip that will locate Spark on the system and import it as a regular library.
# !pip install -q findspark #sparknlp
# # set your spark folder to your system path environment. 
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# import findspark
# findspark.init()
# findspark.find()
# # # OR
# # # # #install pyspark
# # # !pip install -q wheel
# # # !pip install -q pyspark

'/content/spark-3.3.1-bin-hadoop3'

In [ ]:
!pip install -q pyspark==3.3.0 spark-nlp==4.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.4/448.4 KB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.4 MB/s eta 0:00:00


In [ ]:
import sparknlp

spark = sparknlp.start()
# params =>> gpu=False

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.4
Apache Spark version: 3.3.0


In [ ]:
! cd ~/.ivy2/cache/com.johnsnowlabs.nlp/spark-nlp_2.12/jars && ls -lt

total 43992
-rw-r--r-- 1 root root 45044627 Nov 28 16:37 spark-nlp_2.12-4.2.4.jar


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ford_sent_classifn").config('spark.ui.port', '4050').getOrCreate()
#spark = SparkSession.builder.master("local").appName("pyspark").config('spark.ui.port', '4050').getOrCreate()
#spark=SparkSession.builder.appName("local[*]").getOrCreate()
sc = spark.sparkContext
print("Apache Spark version: ", spark.version)
spark

Apache Spark version:  3.3.0


In [ ]:
#Import Necessary Libraries

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import pyspark.sql.functions as F

In [ ]:
#Data Ingestion & Data Preprocessing

# File location and type
file_location = "/content/spam.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.withColumnRenamed("v1", "label")\
    .withColumnRenamed("v2", "text")

df = df.select(F.regexp_replace('label', '\"', '').alias('label'), 'text')

df = df.where("label in ('ham', 'spam')")

df = df.na.drop()

display(df)

DataFrame[label: string, text: string]

In [ ]:
#Display Unique Label Values

unique_label_values = df.dropDuplicates(["label"]).select("label")
unique_label_values.show()

+-----+
|label|
+-----+
|  ham|
| spam|
+-----+



In [ ]:
#Split Dataset into Training & Testing Dataset

train_ds, test_ds = df.randomSplit(weights=[0.80,0.20], seed=42)

In [ ]:
#Instantiate Pipeline Stages

doc = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
    .setInputCols(["document"])\
    .setOutputCol("sent_embeds")

doc_clf = ClassifierDLModel.pretrained('classifierdl_use_spam', 'en') \
    .setInputCols("sent_embeds") \
    .setOutputCol("class")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_spam download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


In [ ]:
#Build Pipeline

spam_nlp_pipeline = Pipeline().setStages([doc, use, doc_clf])

In [ ]:
from sparknlp.base import LightPipeline
light_pipeline = LightPipeline(spam_nlp_pipeline.fit(spark.createDataFrame([['']]).toDF("text")))

# fullannotations = light_pipeline.fullAnnotate("Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/1234 to claim now.")
# print(fullannotations)
annotations = light_pipeline.annotate("Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/1234 to claim now.")
annotations

# https://nlp.johnsnowlabs.com/2021/01/09/classifierdl_use_spam_en.html
# document_assembler = DocumentAssembler()\
#   .setInputCol("text")\
#   .setOutputCol("document")
# use = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
#   .setInputCols(["document"])\
#   .setOutputCol("sentence_embeddings")
# document_classifier = ClassifierDLModel.pretrained('classifierdl_use_spam', 'en') \
#   .setInputCols(["document", "sentence_embeddings"]) \
#   .setOutputCol("class")
# nlpPipeline = Pipeline(stages=[document_assembler, use, document_classifier])
# light_pipeline = LightPipeline(nlp_pipeline.fit(spark.createDataFrame([['']]).toDF("text")))

# annotations = light_pipeline.fullAnnotate('Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/1234 to claim now.')


{'document': ["Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/1234 to claim now."],
 'sent_embeds': ["Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/1234 to claim now."],
 'class': ['spam']}

In [ ]:
#Fit/Train Model

spam_nlp_model = spam_nlp_pipeline.fit(train_ds)

In [ ]:
#Predict Values Based on Testing Dataset

predictions = spam_nlp_model.transform(test_ds)

In [ ]:
#Display Predictions

display(predictions)

DataFrame[label: string, text: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sent_embeds: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, class: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

In [ ]:
#Convert Necessary Features to Pandas DataFrame

predictions_in_pandas = (predictions.select(F.col('text').alias("text"), F.col('label').alias("ground_truth"), F.col('class.result').alias("prediction"))).toPandas()
display(predictions_in_pandas)

,text,ground_truth,prediction
0,&lt;DECIMAL&gt; m but its not a common car he...,ham,[ham]
1,"said kiss, kiss, i can't do the sound effects...",ham,[ham]
2,what number do u live at? Is it 11?,ham,[ham]
3,"""A Boy loved a gal. He propsd bt she didnt min...",ham,[ham]
4,"""Beautiful Truth against Gravity.. Read carefu...",ham,[ham]
...,...,...,...
1066,You will be receiving this week's Triple Echo ...,spam,[spam]
1067,\URGENT! This is the 2nd attempt to contact U!...,spam,[spam]
1068,dating:i have had two of these. Only started a...,spam,[ham]
1069,from www.Applausestore.com MonthlySubscription...,spam,[spam]


In [ ]:
#Instantiate & Display Classification Report

predictions_in_pandas['prediction'] = predictions_in_pandas['prediction'].apply(lambda x : x[0])

report = classification_report(predictions_in_pandas['ground_truth'], predictions_in_pandas['prediction'])
print(report)

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       923
        spam       0.94      0.92      0.93       148

    accuracy                           0.98      1071
   macro avg       0.96      0.95      0.96      1071
weighted avg       0.98      0.98      0.98      1071

